## 2023 农业系统模型与大数据分析第5次实验课代码参考

In [1]:
import numpy as np
import pandas as pd
import random
import itertools
from scipy.signal import savgol_filter
from sklearn import preprocessing
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

### Data Loading

In [2]:
df_raw = pd.read_csv('mp5.csv', header=0, index_col=None)
df_train, df_test = train_test_split(df_raw, test_size=0.2, random_state=42)
X_train, X_test = df_train.iloc[:, :-4], df_test.iloc[:, :-4]
y_train, y_test = df_train.iloc[:, -1], df_test.iloc[:, -1]

### Define Preprocessing Methods

In [3]:
def MS(X_train, X_test):
    scaler = preprocessing.MinMaxScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

def SGF(X_train, X_test):
    X_train_SG = savgol_filter(X_train, 9, 2, mode='nearest') 
    X_test_SG = savgol_filter(X_test, 9, 2, mode='nearest')
    return X_train_SG, X_test_SG

### Define Regressors

In [4]:
def PLSR(X_train, X_test, y_train, y_test):
    pls = PLSRegression(scale=False)
    param_dist = {'n_components':[k for k in range(1,21)]} 
    grid_search = GridSearchCV(estimator = pls, param_grid = param_dist, 
                               cv=5, scoring = 'neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    optim_estimator = grid_search.best_estimator_
    optim_LV = grid_search.best_params_
    y_train_pred = optim_estimator.predict(X_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)
    y_test_pred = grid_search.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)
    return rmse_train, r2_train, rmse_test, r2_test, optim_LV

def PCR(X_train, X_test, y_train, y_test):
    pcr = make_pipeline(PCA(),LinearRegression())
    param_dist = {'pca__n_components':[k for k in range(1,21)]}
    grid_search = GridSearchCV(estimator = pcr, param_grid = param_dist, 
                               cv=5, scoring = 'neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    optim_estimator = grid_search.best_estimator_
    optim_PC = grid_search.best_params_
    y_train_pred = optim_estimator.predict(X_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)
    y_test_pred = grid_search.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)
    return rmse_train, r2_train, rmse_test, r2_test, optim_PC

###  Method Combinations

In [5]:
# Alternative methods
prep = ['', 'SGF', 'MS']
regr = ['PCR', 'PLSR']

# Preprocessing methods and combinations
prep_lst = ['']
prep_comb = [k for k in itertools.combinations(prep, 2)]
for p in prep_comb:
    prep_lst.append('-'.join(p))

# Preprocessing combinations and regressors
prep_regr = list(itertools.product(prep_lst, regr))
prep_regr_lst = []
for k in prep_regr:
    prep_regr_lst.append('-'.join(k))

# Removing redundant dashes'-'
prep_regr_lst_ = []
for n in prep_regr_lst:
    if n[0] == '-':
        n = n[1:]
    prep_regr_lst_.append(n)
prep_regr_lst = prep_regr_lst_

# Showing prep_regr_lst
prep_regr_lst

['PCR',
 'PLSR',
 'SGF-PCR',
 'SGF-PLSR',
 'MS-PCR',
 'MS-PLSR',
 'SGF-MS-PCR',
 'SGF-MS-PLSR']

### Pythonic Coding for Method Combinations (Elective)

In [6]:
prep, regr = ['', 'SGF', 'MS'], ['PCR', 'PLSR']
prep_lst = [''] + [('-'.join(p)) for p in [k for k in itertools.combinations(prep, 2)]]
prep_regr_lst = ['-'.join(k).strip('-') for k in list(itertools.product(prep_lst, regr))]
prep_regr_lst

['PCR',
 'PLSR',
 'SGF-PCR',
 'SGF-PLSR',
 'MS-PCR',
 'MS-PLSR',
 'SGF-MS-PCR',
 'SGF-MS-PLSR']

### Results Evaluation

In [7]:
result_table = pd.DataFrame(columns=['Workflow','Training_RMSE', 'Training_R2',
                                     'Test_RMSE', 'Test_R2', 'Optimized_hp'])
for item in prep_regr_lst:
    X_train_copy, X_test_copy = X_train, X_test
    if 'SGF' in item:
        X_train_copy, X_test_copy = SGF(X_train_copy, X_test_copy)
    if 'MS' in item:
        X_train_copy, X_test_copy = MS(X_train_copy, X_test_copy)   
    if 'PCR' in item:
        training_rmse, training_r2, test_rmse,  test_r2, hp = PCR(X_train_copy, X_test_copy, y_train, y_test)
        result = {'Workflow': item, 'Training_RMSE': training_rmse, 'Training_R2': training_r2, 
                  'Test_RMSE': test_rmse, 'Test_R2': test_r2, 'Optimized_hp': hp}      
        result_table = result_table.append(result, sort=False, ignore_index=True)
        continue
    elif 'PLSR' in item:
            training_rmse, training_r2, test_rmse, test_r2, hp = PLSR(X_train_copy, X_test_copy, y_train, y_test)
            result = {'Workflow': item, 'Training_RMSE': training_rmse, 'Training_R2': training_r2, 
                      'Test_RMSE': test_rmse, 'Test_R2': test_r2, 'Optimized_hp': hp}      
            result_table = result_table.append(result, sort=False, ignore_index=True)

In [8]:
result_table

,Workflow,Training_RMSE,Training_R2,Test_RMSE,Test_R2,Optimized_hp
0,PCR,0.261412,0.882595,0.388191,0.848934,{'pca__n_components': 12}
1,PLSR,0.260358,0.883541,0.366971,0.864998,{'n_components': 9}
2,SGF-PCR,0.261422,0.882586,0.388251,0.848887,{'pca__n_components': 12}
3,SGF-PLSR,0.260749,0.883190,0.366532,0.865321,{'n_components': 9}
4,MS-PCR,0.252618,0.890361,0.379094,0.855931,{'pca__n_components': 15}
5,MS-PLSR,0.239590,0.901379,0.364918,0.866504,{'n_components': 11}
6,SGF-MS-PCR,0.252655,0.890330,0.378780,0.856169,{'pca__n_components': 15}
7,SGF-MS-PLSR,0.240151,0.900916,0.364803,0.866588,{'n_components': 11}
